In [ ]:
from langchain.utilities import SQLDatabase
from langchain.llms import OpenAI
from langchain.agents import create_sql_agent
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from langchain.agents.agent_types import AgentType

In [ ]:
import os
import psycopg2
import pandas as pd
from dotenv import load_dotenv

load_dotenv()
# PostgreSQL database connection parameters
db_params = {
    "dbname": "",
    "user": "",
    "password": "",
    "host": "",
    "port": ""
}

db = SQLDatabase.from_uri("postgresql://udtgpmu99n4j6r:p94fcaffade361eec929012ea55a59a0038a1794ee7363349212b44ebf2e33a1c@cd5gks8n4kb20g.cluster-czrs8kj4isg7.us-east-1.rds.amazonaws.com:5432/d8qdtku8976m7a", include_tables=["smart_home_data"])

# Define SQL query
query= """
SELECT * FROM smart_home_data
"""

with psycopg2.connect(**db_params) as conn:
    with conn.cursor() as cur:
        cur.execute(query)
        data = cur.fetchall()
        column_names = [desc[0] for desc in cur.description]  # Get column names

# Convert to Pandas DataFrame
df = pd.DataFrame(data, columns=column_names)
df

In [ ]:
llm = OpenAI(temperature=0, verbose=True, openai_api_key="")

In [ ]:
agent_executor = create_sql_agent(
    llm=llm,
    toolkit=SQLDatabaseToolkit(db=db, llm=llm),
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

# Prompt 1

In [ ]:
df.columns

In [ ]:
prompt1_df = df.copy()
prompt1_df['month'] = prompt1_df['timestamp'].dt.month
august_appliances = prompt1_df[prompt1_df['month'] == 8][['dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave']]
august_appliances.sum().sort_values(ascending=False)

In [ ]:
prompt1_output = agent_executor.run("In August, which appliance ('dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave') consumed the most energy?")

In [ ]:
prompt1_output

# Prompt 2

In [ ]:
test_output2

In [ ]:
test_output2 = agent_executor.run("I am really trying to save as much money as I can. Since the fridge is consuming the most energy, can I turn it off at some times?")

# Prompt 3

In [ ]:
import plotly.express as px

In [ ]:
grouped_by_month = df.copy()
grouped_by_month['month'] = grouped_by_month['timestamp'].dt.month
grouped_by_month = grouped_by_month[['month', 'dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave']]
grouped_by_month = grouped_by_month.groupby('month').sum()
grouped_by_month

In [ ]:
px.line(grouped_by_month)

In [ ]:
prompt3_output = agent_executor.run("What were the general energy consumption trends month by month?")

# Prompt 4

In [ ]:
appliance_df = df.copy()
appliance_df = appliance_df[['timestamp', 'dishwasher', 'furnace_1', 'furnace_2', 'home_office', 'fridge', 'wine_cellar', 'garage_door', 'barn', 'well', 'microwave']]
appliance_df['date'] = appliance_df['timestamp'].dt.date
appliance_df = appliance_df.drop(columns=['timestamp'])
appliance_df = appliance_df.groupby('date').sum()
appliance_df = appliance_df.sum(axis=1)
appliance_df.sort_values(ascending=False)

In [ ]:
prompt4_output = agent_executor.run("Which day did I consume the most energy among the appliances?")